# Span情報がない平均報酬強化学習

参考：
* [Achieving Tractable Minimax Optimal Regret in Average Reward MDPs](https://arxiv.org/abs/2406.01234)

Average RewardでのRLは一般にSpanの情報や半径の情報が必要になります（UCRL2はいらないけど，リグレットバウンドが半径に依存する）．
今回はSpanが不要なMDPについて学んでみましょう．

表記：
* MDP：$M \in \mathcal{M}$
* ゲイン：$g^\pi(s):=\lim \frac{1}{T} \mathbf{E}_s^\pi\left[R_0+\ldots+R_{T-1}\right]$
* バイアス：$h^\pi:=\lim \sum_{t=0}^{T-1}\left(R_t-g\left(S_t\right)\right)$
* Poisson方程式：$h^\pi+g^\pi=r^\pi+P^\pi h^\pi$
* ベルマン方程式：$L u(s):=\max _{a \in \mathcal{A}(s)}\{r(s, a)+p(s, a) u\}$
  * 今回はWeakly communicatingの設定を考える．つまり，$Lh^* - h^* \in \boldsymbol{R}e$を満たすような$h^*$が存在する．
  * これは任意の方策に対して$r^\pi+P^\pi h^* \leq g^*+h^*$を満たす
* ベルマン誤差：$\Delta^*(s, a):=h^*(s)+g^*(s)-r(s, a)-p(s, a) h^* \geq 0$
* 直径：$D:=\sup _{s \neq s^{\prime}} \inf _\pi \mathbf{E}_s^\pi\left[\inf \left\{t \geq 1: S_t=s^{\prime}\right\}\right]$
* リグレット：
    * $\operatorname{Reg}(T):=T g^*-\sum_{t=0}^{T-1} R_t$
    * $\mathbf{E}[\operatorname{Reg}(T)]=\mathbf{E}\left[\sum_{t=0}^{T-1} \Delta^*\left(X_t\right)\right]+\mathbf{E}\left[h^*\left(S_0\right)-h^*\left(S_T\right)\right]$
* SpanがバウンドされたMDPの集合：$\mathcal{M}_c:=\left\{M \in \mathcal{M}: \exists h^* \in \operatorname{Fix}(L(M)), \operatorname{sp}\left(h^*\right) \leq c\right\}$
  * 下界：$\max _{M \in \mathcal{M}_c} \mathbf{E}^{M, \mathbf{A}}[\operatorname{Reg}(T)]=\Omega(\sqrt{c S A T})$

## PMEVI-DT アルゴリズム

基本的なアイデアはOFUと同じです．OFUは次の楽観的なゲインを計算します：
$$
g^*\left(\mathcal{M}_t\right):=\sup \left\{g^\pi\left(\mathcal{M}_t\right): \pi \in \Pi, \operatorname{sp}\left(g^\pi\left(\mathcal{M}_t\right)\right)=0\right\} \text { with } g^\pi\left(\mathcal{M}_t\right):=\sup \left\{g(\pi, \widetilde{M}): \widetilde{M} \in \mathcal{M}_t\right\}
$$

OFUの更新タイミングはいろいろありますが，今回はDoubling trickを使います．つまり，
$$
N_t\left(S_t, \pi_k\left(S_t\right)\right) \geq 1 \vee 2 N_{t_k}\left(X_t\right)
$$
のタイミングで更新を行います（$X_t = (S_t, A_t)$）．

### Extended Value Iteration (EVI)について

UCRL2など，OFUを実現するためには基本的にEVIを使います．
$(s, a)$-rectangularな不確実集合$\mathcal{M}_t \equiv \prod_{s, a}\left(\mathcal{R}_t(s, a) \times \mathcal{P}_t(s, a)\right)$を作り，次の楽観的な作用素でバイアス関数を更新します：

$$
v_{i+1}(s) \equiv \mathcal{L}_t v_i(s):=\max _{a \in \mathcal{A}(s)} \max _{\tilde{r}(s, a) \in \mathcal{R}_t(s, a)} \max _{\tilde{p}(s, a) \in \mathcal{P}_t(s, a)}\left(\tilde{r}(s, a)+\tilde{p}(s, a) \cdot v_i\right)
$$

そして，スパンが$\operatorname{sp}\left(v_{i+1}-v_i\right)<\epsilon$を満たすまで繰り返すのがEVIです．このとき，$\mathcal{L}_t v_i$を与える方策は
$g^\pi\left(\mathcal{M}_t\right) \geq g^*(\mathcal{M})-\epsilon$を満たすことが知られています．


### Projected Mitigated EVI

基本的に，OFUは$\mathcal{M}_t$の良さによって実現されます．
よって，多くの先行研究は$\mathcal{M}_t$を改善できるように様々な工夫をこらしてました．

今回の論文は，あまり$\mathcal{M}_t$を改善することに固執していません．
* いい感じの挙動をする信頼区間を使い，
* バイアスの推定をして，
* EVIを改善する

ことで，Minimax最適なアルゴリズムを達成するのが今回の論文です．

これを説明するために，何らかの方法で，$h^*$を推定するためのバイアスの信頼区間$\mathcal{H}_t$が与えられているとします．$M\in \mathcal{M}_t$かつ$h^* \in \mathcal{H}_t$が満たされているならば，
* ゲインを最大化して，かつ$h(\pi, \tilde{M}) \in \mathcal{H}_t$を満たすような方策とMDPのペア$(\pi, \tilde{M})$を見つければ，OFUできそうな気がします．

そこで，「Projection」と「Mitigation」の２つのテクニックを使います．

1. Projection: もし$h^* \in \mathcal{H}_t$ならば，OFUで探す最適方策はバイアスが$\mathcal{H}_t$の中に入るものに限定して構いません．そこで，$\Gamma_t: \mathbf{R}^{\mathcal{S}} \rightarrow \mathcal{H}_t$を使ってバイアスを射影します．
2. Mitigation: 一旦ボーナスベースのアルゴリズムについて考えてみましょう．ボーナスベースのアルゴリズムは，
$$
\tilde{p}(s, a) u_i \leq \hat{p}_t(s, a) u_i+\underbrace{\left(p(s, a)-\hat{p}_t(s, a)\right) u_i}_{\leq ボーナス関数}
$$
によって，$推定した遷移\cdot 価値+ボーナス$を使ってOFUを実現します．
今回のアルゴリズムはこれを利用します．
もし$h^* \in \mathcal{H}_t$ならば，$\beta_t(s, a):=\max _{u \in \mathcal{H}_t} \beta_t(s, a, u)$とすれば，$h^*$がわからなくても，$\left(\hat{p}_t(s, a)-p(s, a)\right) h^* \leq \beta_t(s, a)$が成立します．
これを使って，次のEVIを後で利用します．
$$
\mathcal{L}_t^\beta u(s):=\max _{a \in \mathcal{A}(s)} \sup _{\tilde{r}(s, a) \in \mathcal{R}_t(s, a)} \sup _{\tilde{p}(s, a) \in \mathcal{P}_t(s, a)}\left\{\tilde{r}(s, a)+\min \left\{\tilde{p}(s, a) u_i, \hat{p}_t(s, a) u_i+\beta_t(s, a)\right\}\right\}
$$

---

上のProjectionとMitigationを踏まえて，今回のアルゴリズムでは次の「MitigateしてProjection」を繰り返します：

$$\mathfrak{L}_t:=\Gamma_t \circ \mathcal{L}_t^\beta$$

これはCompositionなので，うまく動くかは自明ではありません．しかし，次の定理によって挙動が保証されます：

---

固定した$\beta$を考えます．ここで，次を満たす$\Gamma_t: \boldsymbol{R}^{\mathcal{X}} \to \mathcal{H}_t$を考えましょう：
1. $u \leq v \Rightarrow \Gamma u \leq \Gamma v ;$
2. $\operatorname{sp}(\Gamma u-\Gamma v) \leq \operatorname{sp}(u-v)$
3. $\Gamma(u+\lambda e)=\Gamma u+\lambda e$
4. $\Gamma u \leq u$

このとき，$\mathfrak{L}_t:=\Gamma_t \circ \mathcal{L}_t^\beta$は次を満たします：

書くのがめんどいので省略．
結局Biasのconfidence regionが正しければうまく動く．

---

## バイアスの推定機

今回のバイアスの推定器として，次の制約を組み合わせたものを考えます：
$$\forall s \neq s^{\prime}, \quad \mathfrak{h}(s)-\mathfrak{h}\left(s^{\prime}\right)-c\left(s, s^{\prime}\right) \leq d\left(s, s^{\prime}\right)$$

---

**Bias difference estimator**

**コメント：** この表記めっちゃ分かりづらい．基本的に[RL_AverageReward_estimate-bias.ipynb](RL_AverageReward_estimate-bias.ipynb)と同じなので，そっちを参照．

$s \neq s^{\prime}$が与えられたときに，次の$\left(\tau_i^{s \leftrightarrow s^{\prime}}\right)_{i \geq 0}$をcommute timeの系列と呼ぶ：
* $\tau_{2 i}^{s \leftrightarrow s^{\prime}}:=\inf \left\{t>\tau_{2 i-1}^{s \leftrightarrow s^{\prime}}: S_t=s\right\}$：$s$に訪れる，$\tau_{2 i-1}^{s \leftrightarrow s^{\prime}}$より後の時刻
* $\tau_{2 i+1}^{s \leftrightarrow s^{\prime}}:=\inf \left\{t>\tau_{2 i}^{s \leftrightarrow s^{\prime}}: S_t=s^{\prime}\right\}$：$s'$に訪れる，$\tau_{2 i}^{s \leftrightarrow s^{\prime}}$より後の時刻．$i$は往復回数を表してるっぽい．
* $\tau_{2 -1}^{s \leftrightarrow s^{\prime}}:=-\infty$とする
* $N_t\left(s \leftrightarrow s^{\prime}\right):=\sup \left\{i: \tau_i^{s \leftrightarrow s^{\prime}} \leq t\right\}$：時刻$t$以前に起きた往復回数？（多分supだと思われる）
* $\hat{g}(t):=\frac{1}{t} \sum_{i=0}^{t-1} R_i$：ゲインの推定器

これを使って，Bias difference estimatorを次を満たすような$c_T$として設計します：

$$
N_t\left(s \leftrightarrow s^{\prime}\right) c_T\left(s, s^{\prime}\right)=
\sum_{t=0}^{N_T\left(s \leftrightarrow s^{\prime}\right)-1}(-1)^i \sum_{t=\tau_i^{\tau_s} s^{s^{\prime}}}^{\tau_{i+s^{\prime}}^{s \leftrightarrow s^{\prime}}-1}\left(\hat{g}(T)-R_t\right) .
$$

**実装について:**
次の分解を考えると実装しやすいです：
$$
\begin{aligned}
= 
&\left(\sum_{i=0}^{\text{$s\to s'$の往復回数}}
(-1)^i 
\cdot [\text{$i$の往復時間}]
\right)
\cdot \hat{g}(T)\\
&-\left(\sum_{i=0}^{\text{$s\to s'$の往復回数}}
(-1)^i 
\cdot [\text{$i$中の総報酬}]
\right)
\end{aligned}
$$


---

このとき，次が高確率で成立します：　
任意の$T^{\prime} \leq T$ と$\tilde{g} \geq g^*$，そして指標$c_T\left(s, s^{\prime}\right) \in \mathbf{R}$について，
$$
N_{T^{\prime}}\left(s \leftrightarrow s^{\prime}\right)\left|\underbrace{h^*(s)-h^*\left(s^{\prime}\right)-c_{T^{\prime}}\left(s, s^{\prime}\right)}_{"真のバイアスの差分"と"事前知識"の差}\right| \leq \underbrace{3 \operatorname{sp}\left(h^*\right)}_{無視できそう}+\left(1+\operatorname{sp}\left(h^*\right)\right) \sqrt{8 T \log \left(\frac{2}{\delta}\right)}+\underbrace{2 \sum_{t=0}^{T^{\prime}-1}\left(\tilde{g}-R_t\right)}_{リグレット}
$$

$\operatorname{sp}$は基本的に未知なので，$c_0:=T^{1 / 5}$で近似します．

まとめると，次の流れでバイアスの推定をします：
* バイアス推定器：$c_T\left(s, s^{\prime}\right)=\frac{1}{N_t\left(s \leftrightarrow s^{\prime}\right)} \sum_{t=0}^{N_T\left(s \leftrightarrow s^{\prime}\right)-1}(-1)^i \sum_{t=\tau_i^{\tau_s} s^{s^{\prime}}}^{\tau_{i+s^{\prime}}^{s \leftrightarrow s^{\prime}}-1}\left(\hat{g}(T)-R_t\right)$
* optimisticなゲインを計算（過去のゲインから一番小さいのを取ってくる？）：$\tilde{g} \leftarrow \min _{k<K(t)} \mathfrak{g}_k$
* リグレットの推定：$B_0 \leftarrow t \tilde{g}-\sum_{i=0}^{t-1} R_i$
* $\ell \leftarrow \sqrt{8 T \log \left(\frac{2}{\delta}\right)}, c_0 \leftarrow T^{\frac{1}{5}}$
* バイアス推定器の誤差：$d_t\left(s, s^{\prime}\right) \equiv \operatorname{error}\left(c_t, s, s^{\prime}\right):=\frac{3 c_0+\left(1+c_0\right)(1+\ell)+2 B_0}{N_t\left(s \leftrightarrow s^{\prime}\right)}$


## リグレット解析

まず，OptimismとConfidence boundを使ってリグレットを変形しましょう．最初のステップはUCRL2と同じです：　
$$
\operatorname{Reg}(T) 
\underbrace{\leq}_{\text{optimism}} \sum_k \sum_{t=t_k}^{t_{k+1}-1}\left(\mathfrak{g}_k-R_t\right) 
\underbrace{\leq}_{\text{confidence bound}} \sum_k \sum_{t=t_k}^{t_{k+1}-1}\left(\mathfrak{g}_k-\tilde{r}_k\left(X_t\right)\right)+\mathrm{O}\left(\sqrt{S A T \log \left(\frac{T}{\delta}\right)}\right)
$$

以降，一項目を$\tilde{B}(T):=\sum_\alpha \sum_{t=t_k}^{t_{k+1}-1}\left(\mathfrak{g}_k-\tilde{r}_k\left(X_t\right)\right)$と置きましょう．こいつをバウンドします．まず，Posson方程式（$\mathfrak{h}_k+\mathfrak{g}_k=\tilde{r}_k+\tilde{P}_k \mathfrak{b}_k$）を使って，

$$
\tilde{B}(T)=\sum_k \sum_{t=t_k}^{t_{k+1}-1}\left(\tilde{p}_k\left(S_t\right)-e_{S_t}\right) \mathrm{b}_k
$$

を得ます．こいつを[RL_AverageReward_estimate.ipynb](RL_AverageReward_estimate-bias.ipynb)でやったように分解します：

$$
\sum_{t=t_k}^{t_{k+1}-1}(\underbrace{\left(p_k\left(S_t\right)-e_{S_t}\right) \mathfrak{b}_k}_{\text {navigation error (1k) }}+\underbrace{\left(\hat{p}_k\left(S_t\right)-p_k\left(S_t\right)\right) h^*}_{\text {empirical bias error (2k) }}+\underbrace{\left(\tilde{p}_k\left(S_t\right)-\hat{p}_k\left(S_t\right)\right) \mathfrak{b}_k}_{\text {optimistic overshoot }(3 k)}+\underbrace{\left(\hat{p}_k\left(S_t\right)-p_k\left(S_t\right)\right)\left(\mathrm{b}_k-h^*\right)}_{\text {second order error (4k) }})
$$

TODO: 後はそれぞれをバウンドする


## 実験

![river-swim](figs/river-swim.png)

In [1]:
import numpy as np
import jax.numpy as jnp
from jax.random import PRNGKey
import jax
from typing import NamedTuple, Optional

key = PRNGKey(0)

S = 3  # 状態集合のサイズ
A = 2  # 行動集合のサイズ．LEFTが0, RIGHTが1とします
S_set = jnp.arange(S)  # 状態集合
A_set = jnp.arange(A)  # 行動集合


# 報酬行列（論文中では確率的ですが，今回は面倒なので決定的にします）
rew = np.zeros((S, A))
rew[0, 0] = 0.05
rew[-1, 1] = 0.95
rew = jnp.array(rew)
assert rew.shape == (S, A)


# 遷移確率行列
P = np.zeros((S, A, S))
for s in range(1, S-1):
    P[s, 0, s-1] = 1  # LEFT
    P[s, 1, s-1] = 0.05  # RIGHT
    P[s, 1, s] = 0.6  # RIGHT
    P[s, 1, s+1] = 0.35  # RIGHT

# at s1
P[0, 0, 0] = 1  # LEFT
P[0, 1, 0] = 0.6  # RIGHT
P[0, 1, 1] = 0.4  # RIGHT
P[-1, 0, -2] = 1  # LEFT
P[-1, 1, -2] = 0.05  # RIGHT
P[-1, 1, -1] = 0.95  # RIGHT

P = P.reshape(S, A, S)
P = jnp.array(P)
np.testing.assert_allclose(P.sum(axis=-1), 1, atol=1e-6)  # ちゃんと確率行列になっているか確認します

class MDP(NamedTuple):
    S_set: jnp.array  # 状態集合
    A_set: jnp.array  # 行動集合
    rew: jnp.array  # 報酬行列
    P: jnp.array  # 遷移確率行列

    @property
    def S(self) -> int:  # 状態空間のサイズ
        return len(self.S_set)

    @property
    def A(self) -> int:  # 行動空間のサイズ
        return len(self.A_set)


mdp = MDP(S_set, A_set, rew, P)

print("状態数：", mdp.S)
print("行動数：", mdp.A)

状態数： 3
行動数： 2


In [2]:
from scipy.optimize import linprog
import numpy as np


def solve_optimistic_PV(count_next_SAS: np.ndarray, mdp: MDP, V: np.ndarray, tk: int, delta: float):
    def bonus(count_next_S: np.ndarray):
        # 論文のパラメータはちょっといじります
        return np.sqrt(np.log(2 * mdp.A * tk / delta) / np.maximum(1, count_next_S.sum()))
        # return np.sqrt(14*mdp.S*np.log(2 * mdp.A * tk / delta) / np.maximum(1, count_next_S.sum()))

    def solve_per_sa(count_next_S):
        est_P_sa = count_next_S / np.maximum(1, count_next_S.sum())

        c = np.hstack([-V, np.zeros(S)])  # maximize PV
        A_ub = np.hstack([np.eye(mdp.S), -np.eye(mdp.S)])
        nA_ub = np.hstack([-np.eye(mdp.S), -np.eye(mdp.S)])
        tA_ub = np.hstack([np.zeros(mdp.S), np.ones(mdp.S)])
        A_ub = np.vstack([A_ub, nA_ub, tA_ub])
        b_ub = np.hstack([est_P_sa, -est_P_sa, np.array([bonus(count_next_S)])])

        A_eq = np.hstack([np.ones(mdp.S), np.zeros(mdp.S)]).reshape(1, -1)  # 総和が1になる制約
        b_eq = np.array([1.0])  # 総和は1
        res = linprog(c, A_ub, b_ub, A_eq, b_eq, bounds=(0, None)) 
        return -res.fun

    PV = np.zeros((mdp.S, mdp.A))
    for s in range(mdp.S):
        for a in range(mdp.A):
            count_next_S = count_next_SAS[s, a]
            PV[s, a] = solve_per_sa(count_next_S)
    return jnp.array(PV)
    

ref_state = 0 

def ExtendedValueIteration(count_SAS: jnp.ndarray, mdp: MDP, tk: int, delta: float = 0.9, tol: float = 1e-5) -> jnp.array:
    def condition_fun(nQ_Q):
        nQ, Q = nQ_Q
        nbias = nQ.max(axis=1)  # S -> R
        bias = Q.max(axis=1)  # S -> R
        span_diff = (nbias - bias).max()
        return span_diff > tol

    def body_fun(nQ_Q):
        Q, _ = nQ_Q
        next_v = solve_optimistic_PV(count_SAS, mdp=mdp, V=Q.max(axis=1), tk=tk, delta=delta)
        gain = Q[ref_state].max()
        nQ = mdp.rew + next_v - gain
        return (nQ, Q)

    init_Q = jnp.zeros((mdp.S, mdp.A))
    nQ_Q = (init_Q, init_Q)
    nQ_Q = body_fun(nQ_Q)
    while condition_fun(nQ_Q):
        nQ_Q = body_fun(nQ_Q)
    return nQ_Q[0]


In [40]:
import random

def sample_next_state(mdp: MDP, s: int, a: int):
    """ sample next state according to the transition matrix P
    Args:
        mdp: MDP
        s: int
        a: int
    Returns:
        next_s: int
    """
    probs = np.array(mdp.P[s, a])
    return np.random.choice(mdp.S_set, p=probs)


def sample_eps_greedy_act(mdp: MDP, q_s: np.array, eps: float):
    if random.random() < eps:
        return random.randint(0, mdp.A-1)
    else:
        return q_s.argmax()

In [41]:
# 最適ゲインを評価する用
from itertools import product

@jax.jit
def compute_optimal_bias(mdp: MDP, tol: float = 1e-6) -> jnp.array:
    ref_state = 0 
    def condition_fun(nV_V):
        nV, V = nV_V
        span_diff = (nV - V).max()
        return span_diff > tol

    def body_fun(nV_V):
        V, _ = nV_V
        gain = V[ref_state]
        next_v = mdp.P @ V
        nV = (mdp.rew + next_v).max(axis=1) - gain
        return (nV, V)

    init_V = jnp.zeros((mdp.S))
    nV_V = body_fun((init_V, init_V))
    V, _ = jax.lax.while_loop(condition_fun, body_fun, nV_V)
    return V

optimal_bias = compute_optimal_bias(mdp)
optimal_gain = optimal_bias[ref_state]
optimal_bias_diff = np.zeros((S, S))
for s, ns in product(range(S), range(S)):
    optimal_bias_diff[s, ns] = optimal_bias[s] - optimal_bias[ns]


In [52]:
from tqdm import tqdm

K = 20
init_s = 0

count_SAS = np.zeros((S, A, S))  # 訪問回数

# =====================================
# s -> s' の往復のトリガー．
# Trueならs -> s'の往復中
# Falseなら s -> s' の往復がまだ始まっていない（sに出会うと往復スタート）
com_trigger = np.zeros((S, S), dtype=bool)
com_counts = np.zeros((S, S))  # s -> s' の往復回数
# =====================================
# バイアスの推定器
bias_diff_est = np.zeros((S, S))
com_signed_timesum = np.zeros((S, S))  # バイアスの推定に使う
com_signed_rewsum = np.zeros((S, S))  # バイアスの推定に使う
# =====================================

total_rew = 0
regrets = []
t = 1
s = init_s
for epi in tqdm(range(K)):
    epi_count_SAS = np.zeros((S, A, S))
    Q = ExtendedValueIteration(count_SAS, mdp, t)

    # 探索をします　
    while True:
        # =====================================
        # 行動と報酬の選択
        a = sample_eps_greedy_act(mdp, Q[s], 0.0)
        r = mdp.rew[s, a]

        # リグレットの計算
        total_rew += r
        regret = t * optimal_gain - total_rew
        regrets.append(regret)
        # =====================================
        # 往復中のs -> s'では...sumを更新
        com_sign = (-1)**com_counts * com_trigger  # 往復中のやつにsignをつける
        com_signed_timesum += com_sign  # 往復中のやつは +sign x 1
        com_signed_rewsum += com_sign * r  # 往復中のやつは +sign x r

        # バイアスの推定器の更新
        gain_est = total_rew / (t + 1)
        bias_diff_est = (com_signed_timesum * gain_est - com_signed_rewsum) / (com_counts + 1e-6)
        # =====================================
        # 往復回数の更新
        com_counts[:, s][com_trigger[:, s]] += 1  # trigger==1 & sに出会ったら往復終了
        com_trigger[:, s] = False  # : -> s でtriggerが1のものは往復終了なので-1にする．それ以外は往復が始まってないので-1 （結局全部-1）
        com_trigger[s, :] = True  # s -> : は往復が始まるので1にする
        # =====================================
        # 次状態に遷移
        next_s = sample_next_state(mdp, s, a)
        epi_count_SAS[s, a, next_s] += 1
        s = next_s
        t = t + 1
        # =====================================
        # doubling trick
        if epi_count_SAS[s, a].sum() >= max(1, count_SAS[s, a].sum()):
            break
        # =====================================

    count_SAS += epi_count_SAS

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:04<00:00,  4.33it/s]


In [53]:
com_counts

array([[ 66.,  13.,   9.],
       [ 13., 128.,  47.],
       [  9.,  47., 779.]])

In [58]:
bias_diff_est

Array([[-0.77763534, -0.6554653 , -0.17446387],
       [-1.2828251 ,  0.05487633,  0.71284974],
       [ 3.9274797 , -0.27071762, -0.01300531]], dtype=float32)

In [57]:
optimal_bias_diff

array([[ 0.        , -2.04615235, -4.67692041],
       [ 2.04615235,  0.        , -2.6307683 ],
       [ 4.67692041,  2.6307683 ,  0.        ]])

In [ ]:
# 方策ゲインを評価する用
from itertools import product

@jax.jit
def compute_policy_bias(mdp: MDP, pol: np.ndarray, tol: float = 1e-6) -> jnp.array:
    ref_state = 0 
    def condition_fun(nV_V):
        nV, V = nV_V
        span_diff = jnp.abs(nV - V).max()
        return span_diff > tol

    def body_fun(nV_V):
        V, _ = nV_V
        gain = V[ref_state]
        next_v = mdp.P @ V
        nV = ((mdp.rew + next_v) * pol).sum(axis=1) - gain
        return (nV, V)

    init_V = jnp.zeros((mdp.S))
    nV_V = body_fun((init_V, init_V))
    V, _ = jax.lax.while_loop(condition_fun, body_fun, nV_V)
    return V


In [ ]:
uniform_policy = jnp.ones((S, A)) / A
uniform_bias = compute_policy_bias(mdp, uniform_policy)
uniform_bias_diff = np.zeros((S, S))
for s, ns in product(range(S), range(S)):
    uniform_bias_diff[s, ns] = uniform_bias[s] - uniform_bias[ns]

In [51]:
uniform_bias_diff

array([[ 0.        , -0.15789437, -0.95488614],
       [ 0.15789437,  0.        , -0.79699177],
       [ 0.95488614,  0.79699177,  0.        ]])

In [8]:
import numpy as np

S = 3
T = 8
state_history = np.array([0, 1, 2, 0, 0, 1, 2, 0])
rew_history = np.random.uniform(0, 1.0, size=T)
# s -> s' の往復のトリガー．
# Trueならs -> s'の往復中
# Falseなら s -> s' の往復がまだ始まっていない（sに出会うと往復スタート）
com_trigger = np.zeros((S, S), dtype=bool)
com_counts = np.zeros((S, S))  # s -> s' の往復回数

bias_diff_est = np.zeros((S, S))  # バイアスの推定器
com_signed_timesum = np.zeros((S, S))  # バイアスの推定に使う
com_signed_rewsum = np.zeros((S, S))  # バイアスの推定に使う


for s, r in zip(state_history, rew_history):
    # 往復中のs -> s'では...sumを更新
    com_sign = (-1)**(com_counts) * com_trigger  # 往復中のやつにsignをつける
    com_signed_timesum += com_sign  # 往復中のやつは +sign x 1
    com_signed_rewsum += com_sign * r  # 往復中のやつは +sign x r

    com_counts[:, s][com_trigger[:, s]] += 1  # trigger==1 & sに出会ったら往復終了
    # : -> s でtriggerが1のものは往復終了なので-1にする
    # それ以外は往復が始まってないので-1 （結局全部-1）
    com_trigger[:, s] = False

    # s -> : は往復が始まるので1にする
    com_trigger[s, :] = True


In [9]:
com_signed_timesum
# com_signed_rewsum

array([[ 5., -1., -1.],
       [ 0.,  2.,  0.],
       [ 0.,  2.,  3.]])

In [10]:
state_history

array([0, 1, 2, 0, 0, 1, 2, 0])